In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import boto3
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import math

MAX_NB_WORDS = 100000
SNU_model_All = load_model('SNU_LSTM_Model_All_balance.h5')
SNU_model_pol = load_model('SNU_LSTM_Model_정치_balance.h5')
SNU_model_eco = load_model('SNU_LSTM_Model_경제_balance.h5')
SNU_model_soc = load_model('SNU_LSTM_Model_사회_balance.h5')
SNU_model_others = load_model('SNU_LSTM_Model_기타_balance.h5')

Naver_Comments_All = load_model('Naver_Comments_Model_All.h5')
Naver_Comments_pol = load_model('Naver_Comments_Model_정치.h5')
Naver_Comments_eco = load_model('Naver_Comments_Model_경제.h5')
Naver_Comments_soc = load_model('Naver_Comments_Model_사회.h5')
Naver_Comments_others = load_model('Naver_Comments_Model_기타.h5')

kakao_model = load_model('kakao_LSTM_model.h5')
# SNU_model_shuffle = load_model('SNU_LSTM_Model_shuffle.h5')
similar_model = FastText.load('similar_keyword_model')

class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words


my_Tokenizer = MyTokenizer(Okt())


/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2021-09-24 10:10:57.643 ip-172-31-32-76:1711 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-24 10:10:57.716 ip-172-31-32-76:1711 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [2]:
import pymysql
import sys
import os
import requests
import base64
import json
import logging
import time
import pandas as pd
import csv
import datetime

host = '' # rds endpoint
port = 3306
username = ''
database = ''
password = ''

def connect_RDS(host, port, username, password, database):
    try:
        conn = pymysql.connect(host = host, user = username, password = password, db = database, port = port, use_unicode = True, charset = 'utf8')
        cursor = conn.cursor()
    
    except:
        logging.error('RDS에 연결되지 않았습니다.')
        sys.exit(1)
        
    return conn, cursor

In [3]:
if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'
label_names = ["label"]

def s3_load(category):
    object_key = category
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('cp949')

    # load data
    train_df = pd.read_csv(StringIO(csv_string))
    y_train = train_df[label_names].values

    train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
    max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
    
    return train_df, y_train, max_seq_len

def s3_Comments_load(category):
    object_key = category
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    # load data
    train_df = pd.read_csv(StringIO(csv_string))
    y_train = train_df[label_names].values

    train_df['doc_len'] = train_df['Comments'].apply(lambda words: len(words.split(" ")))
    max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
    
    return train_df, y_train, max_seq_len

train_df_All_, y_train_All_, max_seq_len = s3_load('SNU_All_.csv')
train_df_All_b, y_train_All_b, max_seq_len_All_b = s3_load('SNU_All_b.csv')
train_df_eco, y_train_eco, max_seq_len_eco = s3_load('SNU_경제_b.csv')
train_df_pol, y_train_pol, max_seq_len_pol = s3_load('SNU_정치_b.csv')
train_df_soc, y_train_soc, max_seq_len_soc = s3_load('SNU_사회_b.csv')
train_df_others, y_train_others, max_seq_len_others = s3_load('SNU_기타_b.csv')

train_df_Comments_All, y_train_Comments_All, max_seq_len_Comments = s3_Comments_load('Naver_Comments_All.csv')
train_df_Comments_pol, y_train_Comments_pol, max_seq_len_Comments_pol = s3_Comments_load('Naver_Comments_정치.csv')
train_df_Comments_eco, y_train_Comments_eco, max_seq_len_Comments_eco = s3_Comments_load('Naver_Comments_경제.csv')
train_df_Comments_soc, y_train_Comments_soc, max_seq_len_Comments_soc = s3_Comments_load('Naver_Comments_사회.csv')
train_df_Comments_others, y_train_Comments_All, max_seq_len_Comments_others = s3_Comments_load('Naver_Comments_기타.csv')

In [4]:
train_df_Comments_All

,Comments,label,doc_len
0,"""독립유공자 후손지원, 청년수당, 여성안심택배, 올빼미 버스, 서울로 7017, 도...",1,26
1,"""문정부 집값 잡을 생각이 없구나~~~그래 서민들을 우롱했다~~당신들도 가진자들이다...",1,18
2,"""부정적으로만 보지말시구여 얼마나 살기가 개같으면 코로나인데도 불구하고 모이는지가 ...",0,21
3,"""ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 뭐? 한반도 운전자론? 중립외교? 양쪽에서 이득을 취해?...",0,17
4,"""성범죄는 그나마 개선된거죠 사기죄는 한결같이 제자리...사기 당한 사람들 보면 거...",1,41
...,...,...,...
9610,"""어이 기자양반 그럼 분단비용은 싸게 먹힙니까? 돈으로 추산 안되는 한반도 디스카운...",0,26
9611,"""주휴시간 폐지, 최저임금 차등적용, 세금, 4대보험 본인부담 하게 만들어라.""",0,9
9612,"""연동률 20프로로 낮추어라""",0,3
9613,"""헛다리에 헛발질 자한당 답다""",0,4


**단어 벡터화 및 패딩**

In [5]:
def embedding_padding(train_df):
    raw_docs_train = train_df['document'].tolist()
    num_classes = len(label_names)
    print(num_classes)
    processed_docs_train = []

    for doc in tqdm(raw_docs_train):
        tokens = my_Tokenizer(doc)
        processed_docs_train.append(tokens)

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
    tokenizer.fit_on_texts(processed_docs_train)
    return tokenizer

def Comments_embedding_padding(train_df):
    raw_docs_train = train_df['Comments'].tolist()
    num_classes = len(label_names)
    print(num_classes)
    processed_docs_train = []

    for doc in tqdm(raw_docs_train):
        tokens = my_Tokenizer(doc)
        processed_docs_train.append(tokens)

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
    tokenizer.fit_on_texts(processed_docs_train)
    return tokenizer

tokenizer_All = embedding_padding(train_df_All_b)
tokenizer_pol = embedding_padding(train_df_pol)
tokenizer_soc = embedding_padding(train_df_soc)
tokenizer_eco = embedding_padding(train_df_eco)
tokenizer_others = embedding_padding(train_df_others)

tokenizer_Comments_All = Comments_embedding_padding(train_df_Comments_All)
tokenizer_Comments_pol = Comments_embedding_padding(train_df_Comments_pol)
tokenizer_Comments_eco = Comments_embedding_padding(train_df_Comments_eco)
tokenizer_Comments_soc = Comments_embedding_padding(train_df_Comments_soc)
tokenizer_Comments_others = Comments_embedding_padding(train_df_Comments_others)

  0%|          | 0/1441 [00:00<?, ?it/s]

1


  1%|          | 4/509 [00:00<00:23, 21.30it/s]

1


  8%|▊         | 41/490 [00:00<00:01, 401.59it/s]

...

  0%|          | 0/2934 [00:00<?, ?it/s]

1


  0%|          | 0/1537 [00:00<?, ?it/s]

1


100%|██████████| 1537/1537 [00:22<00:00, 67.94it/s]


In [6]:
# word_seq_train = tokenizer_All.texts_to_sequences(processed_docs_train)
# word_index = tokenizer_All.word_index
# print('dictionary size :', len(word_index))

# word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
# word_seq_train

In [7]:
# 카카오 응답 데이터 전처리
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
re.sub(r'[^a-zA-Z ]', '', text) #알파벳과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
X_train = []
for sentence in tqdm(train_data['document']):
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer2.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer2.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 1
tokenizer_kakao = Tokenizer(vocab_size) 
tokenizer_kakao.fit_on_texts(X_train)
X_train = tokenizer_kakao.texts_to_sequences(X_train)

max_len = 30

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':
100%|██████████| 145393/145393 [23:30<00:00, 103.05it/s]


**사용자에게 받은 데이터 전처리 테스트**

In [8]:
# response = ['게르마늄 팔찌, 과학적으로 효능 있을까?', '빌려쓰는 IT서비스 클라우드컴퓨팅, 실제 비용절감 효과 있나', '"심장마비 왔을 때 강하게 기침하면 막을 수 있다"	']
# tokens_response = []

# for i in range(len(response)):
#     tokens = my_Tokenizer(response[i])
#     tokens_response.append(tokens)
# print(tokens_response)

# word_seq_response = tokenizer.texts_to_sequences(tokens_response)
# print(word_seq_response)
# word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)
# print(word_seq_response.shape)

# score_list = []
# for sco in range(len(word_seq_response)):
#     word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
#     print(word_seq_response_to_score)
#     score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
# print(score_list)

**입력 데이터 처리 함수**

In [47]:
# ===========================================================================================================================
# 카카오톡 응답 데이터 score

def kakao_data_score(response):
    objectivity = ''
    response = okt.morphs(response, stem=True) # 토큰화
    response = [word for word in response if not word in stopwords] # 불용어 제거
    encoded = tokenizer_kakao.texts_to_sequences([response]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(kakao_model.predict(pad_new)) # 예측
    if(score > 0.8 or score < 0.2):
        objectivity = '객관성이 매우 낮습니다.'
    elif(score > 0.65 or score < 0.35):
        objectivity = '객관성이 낮습니다.'
    else:
        objectivity = '객관성이 높습니다.'
    
    return objectivity


# ===========================================================================================================================
# 단어 토큰화 및 빈도순 정렬 후 추출

def tfidf_Vectorizer(response):
    response_list = []
    response_list.append(response)
    tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_Tokenizer, min_df=1) # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
    X = tfidf_Vectorizer.fit_transform(response_list).toarray()
#     print(X.shape)    # X(2차원 배열)의 행,열 수를 출력.
#     print(tfidf_Vectorizer.vocabulary_)   # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.


    #pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력
    count = X.sum(axis=0)    # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
    word_count = pd.DataFrame({
        '단어' : tfidf_Vectorizer.get_feature_names(),
        '빈도' : count.flat
    })
    sorted_df = word_count.sort_values('빈도', ascending=False)
#     print(sorted_df.head(10), "\n")

    word_ = list(np.array(sorted_df['단어'].tolist()))
    return(word_)


# ===========================================================================================================================
# 네이버 댓글 분석

def Naver_Score(response, tokenizer, tokenizer_Comments, SNU_Model, Naver_Comments_Model):
    response = re.compile("[^A-Za-z0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]+").sub('',response)
    response = okt.morphs(response, stem=True) # 토큰화
    response = [word for word in response if not word in stopwords] # 불용어 제거
    response = ' '.join(response)
    
    word_ = tfidf_Vectorizer(response)
    word__ = ""
    if len(word_) <= 6:
        word__ = response
    else:
        for i in range(1, min(len(word_), 7)):
            word__ += word_[i]
            word__ += " "
    
    baseurl = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="

    url = baseurl + urllib.parse.quote_plus(word__)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find_all(class_ = 'news_tit')

    title_list = []
    link_list = []

    for i in title:
        title_list.append(i.attrs['title'])
        link_list.append(i.attrs['href'])

# ================================================================

    tokens_response = []
    SNU_score_list = []

    for i in range(len(title_list)):
        tokens = my_Tokenizer(title_list[i])
        tokens_response.append(tokens)

    word_seq_response = tokenizer.texts_to_sequences(tokens_response)
    word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)

    for sco in range(len(title_list)):
        word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
        SNU_score_list.append(float(SNU_Model.predict(word_seq_response_to_score)))
        
    df_qq = []
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}

    allComments = []
    def create_soup(url):
        res = requests.get(url, headers=headers)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup

    Comments_score_list = []
    # 네이버 뉴스 url을 입력합니다.
    url1 = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}".format(word__)
    soup = create_soup(url1)
    url_check = []
    for i in range(1, 30):
        List = []
        try:
            url = soup.select_one("#sp_nws{} > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-of-type(2)".format(i))['href']
            url_check.append(url)
            oid = url.split("oid=")[1].split("&")[0] #422
            aid = url.split("aid=")[1] #0000430957
            page = 1
            header = {
                "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
                "referer": url,
            }
            # time.sleep(random.randint(1,2))

            while True:
                c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(
                    page) + "&refresh=false&sort=FAVORITE"
                # 파싱하는 단계입니다.
                r = requests.get(c_url, headers=header)
                cont = BeautifulSoup(r.content, "html.parser")
                total_comm = str(cont).split('comment":')[1].split(",")[0]

                match = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
                # 댓글을 리스트에 중첩합니다.
                try:
                    for b in range(0,5):
                        List.append(match[b])
                except:
                    pass
                break

            tokens_response = []
            score_list = []

            for i in range(len(List)):
                tokens = my_Tokenizer(List[i])
                tokens_response.append(tokens)

            word_seq_response2 = tokenizer_Comments.texts_to_sequences(tokens_response)
            word_seq_response2 = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len_Comments)

            for sco in range(len(List)):
                word_seq_response_to_score2 = word_seq_response2[sco].reshape(1,max_seq_len_Comments)
                score_list.append(float(Naver_Comments_Model.predict(word_seq_response_to_score2)))

            Comments_score_list.append(np.mean(score_list))

        except:
            pass
        if len(url_check) == len(title_list):
            break
    
    print(SNU_score_list)
    print(Comments_score_list)
    a = [score * 0.5 for score in SNU_score_list]
#     b = [score * 0.5 if math.isnan(score)==False else score2 * 0.5 for score2 in SNU_score_list for score in Comments_score_list]
    b = [score * 0.5 for score in Comments_score_list]
    for i in range(len(b)):
        if math.isnan(b[i]):
            b[i] = a[i]
            
    if len(b) < len(a):
        len_b = len(b)
        for i in range(len(a)-len(b)):
            b.append(a[i+len_b])
    elif len(a) < len(b):
        len_a = len(a)
        for i in range(len(b)-len(a)):
            a.append(b[i+len_a])
        
    score_list = list(map(lambda x,y:x+y, a,b))
    print('-'*50)
    print(a)
    print(b)
    print('-'*50)
    print(score_list)
    
    return title_list, link_list, score_list

# ===========================================================================================================================
# 연관키워드 추출

def similar_keyword(word_):
    similar_list = []

    if len(word_) == 1:
        similar_keywords = similar_model.wv.most_similar(word_[0])

        for i in range(len(similar_keywords)):
            similar_list.append(similar_keywords[i][0])
    return similar_list


# ===========================================================================================================================
# MySQL 데이터베이스에 응답데이터, 시간 저장

def MySQL_data_save(response):
    global cursor, conn
    conn, cursor = connect_RDS(host, port, username, password, database)

    now = datetime.datetime.now()
    Korea_DatetimeNow = now + datetime.timedelta(hours = 9)
    DatetimeNow = Korea_DatetimeNow.strftime('%Y-%m-%d %H:%M:%S')

    try:
        with conn.cursor() as cursor:
            query = "INSERT INTO factcheck.article_data (article_data, date) VALUES(%s, %s)"
            val = (response, DatetimeNow)
            cursor.execute(query, val)
            conn.commit()

    finally:
        conn.close()
        
        
# ===========================================================================================================================
# MySQL 데이터베이스에 키워드 랭킹 반영

def keyword_rank(word_):
    rank_list = []
    conn, cursor = connect_RDS(host, port, username, password, database)

    try:
        with conn.cursor() as cursor:
            cursor.execute('SELECT keyword FROM keyword_rank')
            row = cursor.fetchall()

        for i in range(len(row)):
            rank_list.append(row[i][0])

        with conn.cursor() as cursor:
            for i in range(0,1):
                if word_[i] in rank_list:
                    query = "UPDATE keyword_rank SET count = count + 1 WHERE keyword = (%s)"
                    val = (word_[i])
                    cursor.execute(query, val)
                    conn.commit()
                else:
                    query = "INSERT INTO factcheck.keyword_rank (keyword, count) VALUES(%s, 1)"
                    val = (word_[i])
                    cursor.execute(query, val)
                    conn.commit()

        top10_list = []
        with conn.cursor() as cursor:
            cursor.execute("SELECT keyword FROM keyword_rank ORDER BY count DESC LIMIT 10")
            row_top = cursor.fetchall()
            for i in range(len(row_top)):
                top10_list.append(row_top[i][0])

    finally:
        conn.close()
    return top10_list


# ===========================================================================================================================
# SNU 기사 검색      

def SNU_Search(word_):
    title_list2 = []
    link_list2 = []
    score_list2 = []

    try:
#             str_expr = "document.str.contains('{}', case=False)".format(word_[0]) and "document.str.contains('{}', case=False)".format(word_[1])
        str_expr = "document.str.contains('{}', case=False)".format(word_[0])

        df_q = train_df_All_.query(str_expr, engine="python")

        title = list(np.array(df_q['document'].tolist()))
        score_SNU = list(np.array(df_q['label'].tolist()))
        link = list(np.array(df_q['link'].tolist()))

        for i in range(len(title)):
            title_list2.append(title[i])
            link_list2.append(link[i])
            score_list2.append(score_SNU[i])

    except:
        pass
    return title_list2, score_list2


# ===========================================================================================================================
# 네이버 뉴스 검색

def Naver_Search(response, tokenizer, SNU_model):
    baseurl = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="

    url = baseurl + urllib.parse.quote_plus(response)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find_all(class_ = 'news_tit')

    title_list = []
    link_list = []

    for i in title:
        title_list.append(i.attrs['title'])
        link_list.append(i.attrs['href'])

# ================================================================

    tokens_response = []
    score_list = []

    for i in range(len(title_list)):
        tokens = my_Tokenizer(title_list[i])
        tokens_response.append(tokens)

    word_seq_response = tokenizer.texts_to_sequences(tokens_response)
    word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)

    for sco in range(len(title_list)):
        word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
        score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
    
    return title_list, link_list, score_list

**Flask 웹 서버**

In [ ]:
#-*-coding:utf-8-*-

from flask import Flask, request
import json
from urllib import parse

app = Flask(__name__)

@app.route('/politics', methods = ['POST'])
def Article_keyword_politics():
    if request.method == 'POST':
        params = request.get_data()
        params_decode = str(params.decode('utf-8'))
        urldecode = parse.unquote(params_decode)
        
        # 사용자 요청 데이터
        response = urldecode.split('=')[1]
        
        word_ = tfidf_Vectorizer(response)
        
        similar_list = similar_keyword(word_)
        MySQL_data_save(response)
        top10_list = keyword_rank(word_)
        
        SNU_title, SNU_score = SNU_Search(word_)
#         Naver_title, Naver_link, Naver_score = Naver_Search(response, tokenizer_pol, SNU_model_pol)
        Naver_title, Naver_link, Naver_score = Naver_Score(response, tokenizer_pol, tokenizer_Comments_pol, SNU_model_pol, Naver_Comments_pol)
        
        if len(similar_list) == 0:
            similar_list.append('연관 검색어가 없습니다.')
        if len(SNU_title) == 0:
            SNU_title.append('관련 뉴스 결과가 없습니다.')
        if len(SNU_score) == 0:
            SNU_score.append('x')
        if len(Naver_title) == 0:
            Naver_title.append('관련 뉴스 결과가 없습니다.')
        if len(Naver_link) == 0:
            Naver_link.append('x')
        if len(Naver_score) == 0:
            Naver_score.append('x')
            
        data_ = {
            'response':response,
            'similar_list':similar_list,
            'SNU_title':SNU_title,
            'SNU_score':SNU_score,
            'Naver_title':Naver_title,
            'Naver_link':Naver_link,
            'Naver_score':Naver_score,
            'top10_list':top10_list
            }
        
        return json.dumps(data_, ensure_ascii=False)
    
@app.route('/economy', methods = ['POST'])
def Article_keyword_economy():
    if request.method == 'POST':
        params = request.get_data()
        params_decode = str(params.decode('utf-8'))
        urldecode = parse.unquote(params_decode)
        
        # 사용자 요청 데이터
        response = urldecode.split('=')[1]
        
        word_ = tfidf_Vectorizer(response)
        
        similar_list = similar_keyword(word_)
        MySQL_data_save(response)
        top10_list = keyword_rank(word_)
        
        SNU_title, SNU_score = SNU_Search(word_)
        Naver_title, Naver_link, Naver_score = Naver_Search(response, tokenizer_eco, SNU_model_eco)
        Naver_title, Naver_link, Naver_score = Naver_Score(response, tokenizer_eco, tokenizer_Comments_eco, SNU_model_eco, Naver_Comments_eco)
        
        if len(similar_list) == 0:
            similar_list.append('연관 검색어가 없습니다.')
        if len(SNU_title) == 0:
            SNU_title.append('관련 뉴스 결과가 없습니다.')
        if len(SNU_score) == 0:
            SNU_score.append('x')
        if len(Naver_title) == 0:
            Naver_title.append('관련 뉴스 결과가 없습니다.')
        if len(Naver_link) == 0:
            Naver_link.append('x')
        if len(Naver_score) == 0:
            Naver_score.append('x')
            
        data_ = {
            'response':response,
            'similar_list':similar_list,
            'SNU_title':SNU_title,
            'SNU_score':SNU_score,
            'Naver_title':Naver_title,
            'Naver_link':Naver_link,
            'Naver_score':Naver_score,
            'top10_list':top10_list
            }
        
        return json.dumps(data_, ensure_ascii=False)
    
@app.route('/society', methods = ['POST'])
def Article_keyword_society():
    if request.method == 'POST':
        params = request.get_data()
        params_decode = str(params.decode('utf-8'))
        urldecode = parse.unquote(params_decode)
        
        # 사용자 요청 데이터
        response = urldecode.split('=')[1]
        
        word_ = tfidf_Vectorizer(response)
        
        similar_list = similar_keyword(word_)
        MySQL_data_save(response)
        top10_list = keyword_rank(word_)
        
        SNU_title, SNU_score = SNU_Search(word_)
        Naver_title, Naver_link, Naver_score = Naver_Search(response, tokenizer_soc, SNU_model_soc)
        Naver_title, Naver_link, Naver_score = Naver_Score(response, tokenizer_soc, tokenizer_Comments_soc, SNU_model_soc, Naver_Comments_soc)

        if len(similar_list) == 0:
            similar_list.append('연관 검색어가 없습니다.')
        if len(SNU_title) == 0:
            SNU_title.append('관련 뉴스 결과가 없습니다.')
        if len(SNU_score) == 0:
            SNU_score.append('x')
        if len(Naver_title) == 0:
            Naver_title.append('관련 뉴스 결과가 없습니다.')
        if len(Naver_link) == 0:
            Naver_link.append('x')
        if len(Naver_score) == 0:
            Naver_score.append('x')
            
        data_ = {
            'response':response,
            'similar_list':similar_list,
            'SNU_title':SNU_title,
            'SNU_score':SNU_score,
            'Naver_title':Naver_title,
            'Naver_link':Naver_link,
            'Naver_score':Naver_score,
            'top10_list':top10_list
            }
        
        return json.dumps(data_, ensure_ascii=False)
    
@app.route('/others', methods = ['POST'])
def Article_keyword_others():
    if request.method == 'POST':
        params = request.get_data()
        params_decode = str(params.decode('utf-8'))
        urldecode = parse.unquote(params_decode)
        
        # 사용자 요청 데이터
        response = urldecode.split('=')[1]
        
        word_ = tfidf_Vectorizer(response)
        
        similar_list = similar_keyword(word_)
        MySQL_data_save(response)
        top10_list = keyword_rank(word_)
        
        SNU_title, SNU_score = SNU_Search(word_)
        Naver_title, Naver_link, Naver_score = Naver_Search(response, tokenizer_others, SNU_model_others)
        Naver_title, Naver_link, Naver_score = Naver_Score(response, tokenizer_others, tokenizer_Comments_others, SNU_model_others, Naver_Comments_others)

        if len(similar_list) == 0:
            similar_list.append('연관 검색어가 없습니다.')
        if len(SNU_title) == 0:
            SNU_title.append('관련 뉴스 결과가 없습니다.')
        if len(SNU_score) == 0:
            SNU_score.append('x')
        if len(Naver_title) == 0:
            Naver_title.append('관련 뉴스 결과가 없습니다.')
        if len(Naver_link) == 0:
            Naver_link.append('x')
        if len(Naver_score) == 0:
            Naver_score.append('x')
            
        data_ = {
            'response':response,
            'similar_list':similar_list,
            'SNU_title':SNU_title,
            'SNU_score':SNU_score,
            'Naver_title':Naver_title,
            'Naver_link':Naver_link,
            'Naver_score':Naver_score,
            'top10_list':top10_list
            }
        
        return json.dumps(data_, ensure_ascii=False)

@app.route('/kakao', methods = ['POST'])
def kakao_():
    if request.method == 'POST':
        params = request.get_data()
        params_decode = str(params.decode('utf-8'))
        urldecode = parse.unquote(params_decode)
        
        # 사용자 요청 데이터
        response = urldecode.split('=')[1]
        
        objectivity = kakao_data_score(response)
        word_ = tfidf_Vectorizer(response)
        
        similar_list = similar_keyword(word_)
        MySQL_data_save(response)
        top10_list = keyword_rank(word_)
        
        SNU_title, SNU_score = SNU_Search(word_)
        Naver_title, Naver_link, Naver_score = Naver_Search(response, tokenizer_All, SNU_model_All)
#         Naver_title, Naver_link, Naver_score = Naver_Score(response, tokenizer_All, tokenizer_Comments_All, SNU_model_All, Naver_Comments_All)

        if len(similar_list) == 0:
            similar_list.append('연관 검색어가 없습니다.')
        if len(SNU_title) == 0:
            SNU_title.append('관련 뉴스 결과가 없습니다.')
        if len(SNU_score) == 0:
            SNU_score.append('x')
        if len(Naver_title) == 0:
            Naver_title.append('관련 뉴스 결과가 없습니다.')
        if len(Naver_link) == 0:
            Naver_link.append('x')
        if len(Naver_score) == 0:
            Naver_score.append('x')
        
        data_ = {
            'response':response,
            'objectivity':objectivity,
            'similar_list':similar_list,
            'SNU_title':SNU_title,
            'SNU_score':SNU_score,
            'Naver_title':Naver_title,
            'Naver_link':Naver_link,
            'Naver_score':Naver_score,
            'top10_list':top10_list
            }
        
        print('입력 데이터 : \n', response)
        print('\n')
        print('객관성 level : \n', objectivity)
        print('\n')
        print('연관 검색어 : \n', similar_list)
        print('\n')
        print('네이버 뉴스 제목 : \n', Naver_title)
        print('\n')
        print('네이버 뉴스 링크 : \n', Naver_link)
        print('\n')
        print('네이버 뉴스 score : \n', Naver_score)
        print('\n')
        try:
            print('SNU 기사 제목 : \n', SNU_title)
            print('\n')
    #         print('SNU 기사 링크 : ', link_list2)
            print('SNU 기사 score : \n', SNU_score)
            print('\n')
        except:
            print('SNU 기사가 없습니다. \n')
        print('인기키워드 랭킹 : \n', top10_list)
        print('\n')
        print('\n')
        
        return json.dumps(data_, ensure_ascii=False)
    
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port = 8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
125.133.174.241 - - [24/Sep/2021 11:31:32] "POST /politics HTTP/1.1" 200 -


[0.18024177849292755, 0.6721988916397095, 0.36808374524116516, 0.47105246782302856]
[nan, 0.41467059403657913]
--------------------------------------------------
[0.09012088924646378, 0.33609944581985474, 0.18404187262058258, 0.23552623391151428]
[0.09012088924646378, 0.20733529701828957, 0.18404187262058258, 0.23552623391151428]
--------------------------------------------------
[0.18024177849292755, 0.5434347428381443, 0.36808374524116516, 0.47105246782302856]
